In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
from dotenv import load_dotenv
load_dotenv("../.env")  # take environment variables
PROJECT_ROOT = os.environ.get("PROJECT_ROOT")
sys.path.append(PROJECT_ROOT)

import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from datetime import datetime
import random

import utils.PATHS as PATHS
import utils.utils as utils
# import utils.emr_utils as emr_utils
# import utils.load_utils as load_utils

# Lipid-lowering treatment criteria for scaling of LDL-C values

## Notes

* Drug dispensed data for each patient is based on the `Generic Drug Name` column
* Currently the `Drug Prescribed Date From` is used as reference as `Drug Prescribed Date To` may go beyond the `Index Date` from the previous notebook.
* This notebook proceeds as follows:
   1. Creation of a patient list according to lipid-lowering treatment (LLT) criteria
   2. Scaling the LDL-C values accordingly, i.e., dividing LDL-C values 0.7 (confirm if this is divide or multiple like a scaling factor)
   3. Segmentation of cohort according to scaled LDL-C thresholds
      - Cases: LDL-C >= `4.9`
      - Unknown: `3.36` < LDL-C < `4.9`
      - Controls: LDL-C <= `3.36`

## 1. LLT Data Preprocessing

In [3]:
dd_fp_list = glob(os.path.join(PATHS.DRUG_DISPENSED, "*.csv"))

# general labs dataframe
df_list = [pd.read_csv(path, low_memory=False) for path in tqdm(dd_fp_list)]
df = pd.concat(df_list, ignore_index=True)

drug_name_col = "Generic Drug Name"
pid_col = "Patient ID"
date_col = "Drug Prescribed Date From"
backup_date_col = "Drug Dispensed Date From" # has more weird values
ref_date_col = "Drug Reference Date"

# prep datetime cols
df[date_col] = pd.to_datetime(df[date_col])
df[backup_date_col] = pd.to_datetime(df[backup_date_col])

100%|███████████████████████████████████████████████████████| 48/48 [03:05<00:00,  3.87s/it]


In [4]:
tqdm.pandas()

# RESOLVE OPTION
def resolve_date(row):
    """Use prescribed date when dispensed date is not available."""
    if np.isnat(np.datetime64(str(row[date_col]))):
        return row[backup_date_col]
    return row[date_col]

df[ref_date_col] = df.progress_apply(lambda row: resolve_date(row), axis=1)

100%|███████████████████████████████████████| 20132904/20132904 [03:21<00:00, 100041.77it/s]


In [5]:
# DROP OPTION

# df["Drug Reference Date"] = df[date_col]
# df = df.dropna(subset=["Drug Reference Date"])

In [6]:
# see utils.llt_meds for list of relevant prescriptions
llt = df[df[drug_name_col].str.lower().isin(utils.llt_meds)]

In [7]:
# # Inspect weird dates
# grouper = df.groupby(df[backup_date_col].dt.year)
# grouper.size()

## 2. Load the LDL-C values

In [8]:
path = os.path.join(PROJECT_ROOT, "results", "ldlc_valid_18_nontg_index_nosecondary_final.csv")
ldlc_valid_18_nontg_index_nosecondary_final = pd.read_csv(path)

In [9]:
ldlc_valid_18_nontg_index_nosecondary_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20786 entries, 0 to 20785
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Patient ID  20786 non-null  object 
 1   LDL-C Max   20786 non-null  float64
 2   Index Date  20786 non-null  object 
dtypes: float64(1), object(2)
memory usage: 487.3+ KB


In [10]:
# prep datetime cols
ldlc_valid_18_nontg_index_nosecondary_final["Index Date"] = pd.to_datetime(
    ldlc_valid_18_nontg_index_nosecondary_final["Index Date"]
)

## 3. LLT data filtering based on LDL-C Index Date

In [11]:
# excldue drug data that are too old: over 1 year before the minimum LDLC Index Date
mask = df[ref_date_col] < (ldlc_valid_18_nontg_index_nosecondary_final["Index Date"].min() - pd.Timedelta(days=366))
llt = df[~mask]

# check
display(llt[ref_date_col].min())
display(llt[ref_date_col].max())

Timestamp('2014-11-04 00:00:00')

Timestamp('2024-03-27 00:00:00')

# 4. Scaling the LDL-C Max values

In [12]:
ezetimibe_combis = {
    "atorvastatin": {
        "10mg" : 2.0,
        "20mg" : 2.2,
        "40mg" : 2.2,
    },
    "pravastatin sodium": {
        '20MG': 1.6,
    },
    "simvastatin": {
        "10mg": 1.9,
        "20mg": 2.0,
    },
    "rosuvastatin" : {
        "10MG" : 1.9,
        "20MG" : 2.1,
    },
}

special_factors = {
    "pravastatin sodium": {
        '20MG': 1.3,
    }, # 20MG

    "ezetimibe,simvastatin tab" : {
        "10/10": 1.9,
        "10/20": 2.0,
    }, # 10/10, 10/20

    "ezetimibe": {
        "10mg": 1.2,
    }, # 10mg

    "simvastatin": {
        "10mg": 1.4,
        "20mg": 1.6,
    }, # 10mg, 20mg

    "atorvastatin": {
        "10mg" : 1.6,
        "20mg" : 1.8,
        "40mg" : 2.0,
    }, # 10mg, 40mg, 20mg

    "rosuvastatin": {
        "10MG" : 1.9,
        "20MG" : 2.1,
    }, # 10MG, 20MG

    # # --- NOT IN NUHS DOC --- #
    # "lovastatin": {
    #     "20mg" : None,
    # }, # 20mg

    # "metronidazole, nystatin": {
    #     "500mg": None,
    #     "100,000 unit": None,
    # }, # 500mg, 100,000 unit  

    # "nystatin": {
    #     "100,000IU" : None,
    # }, # 100,000IU
} 

In [13]:
def llt_rescale(row):
    pid = row["Patient ID"]
    index_date = row["Index Date"]
    temp_llt = llt[llt[pid_col] == pid]
    
    ## take only those records which are between 365 days and 42 days of index date
    temp_llt = temp_llt[
        (index_date - temp_llt[ref_date_col] >= pd.Timedelta(days=42)) & 
        (index_date - temp_llt[ref_date_col] <= pd.Timedelta(days=365))
    ]
    
    ## if not empty apply correction else do not
    if (temp_llt.empty):
        factor = 1
    else:
        # sort by date, get the most_recent_date, get all dispensed on same most_recent-date         
        most_recent_date = temp_llt[ref_date_col].sort_values(ascending=False).iloc[0]
        temp_llt = temp_llt[temp_llt[ref_date_col] == most_recent_date]

        # intersections
        prescribed_list = temp_llt[drug_name_col].str.lower().unique().tolist()

        # referenced later in elif block
        special_set = set(prescribed_list).intersection(set(special_factors.keys()))
        
        if len(prescribed_list) == 1:
            drug_key =  prescribed_list[0]
            factor_dict = special_factors.get(drug_key)
            if not factor_dict:
                if drug_key in utils.llt_meds:
                    factor = 1/0.7
                else:
                    factor = 1
            else:
                drug_subset = temp_llt.loc[temp_llt[drug_name_col].str.lower() == drug_key]
                dose_key = drug_subset.iloc[0]["Drug Strength"]
                factor = factor_dict.get(dose_key)
            
        elif len(set(prescribed_list).intersection(set(ezetimibe_combis.keys()))) \
            and ("ezetimibe" in prescribed_list):
            prescribed_list.remove("ezetimibe")
            
            combis_set = set(prescribed_list).intersection(set(ezetimibe_combis.keys()))
            
            # get a random special drug
            drug_key = random.sample(list(combis_set), 1)[0]

            # get dose
            drug_subset = temp_llt.loc[temp_llt[drug_name_col].str.lower() == drug_key]
            dose_key = drug_subset.iloc[0]["Drug Strength"]

            # get factor
            factor = ezetimibe_combis.get(drug_key).get(dose_key)

        # next block assumes ezetimibe cases has already been evaluated
        # non-ezetimibe combinations
        elif len(special_set): 
        
            # get a random special drug
            drug_key = random.sample(list(special_set), 1)[0]

            # get dose
            drug_subset = temp_llt.loc[temp_llt[drug_name_col].str.lower() == drug_key]
            dose_key = drug_subset.iloc[0]["Drug Strength"]

            # get factor
            factor = special_factors.get(drug_key).get(dose_key)

        # default scaling factor on master list of llt medications (see utils.py)
        elif len(set(prescribed_list).intersection(set(utils.llt_meds))):
            factor = 1/0.7

        else:
            factor = 1 
    # display(row["Patient ID"])        
    return row["LDL-C Max"] * factor
    

In [14]:
tqdm.pandas()

ldlc_valid_18_nontg_index_nosecondary_final["Scaled LDL-C Max"] = ldlc_valid_18_nontg_index_nosecondary_final.progress_apply(
    llt_rescale, axis=1
    
)

100%|███████████████████████████████████████████████| 20786/20786 [5:19:51<00:00,  1.08it/s]


In [16]:
# number of cases scaled
(ldlc_valid_18_nontg_index_nosecondary_final["LDL-C Max"] != ldlc_valid_18_nontg_index_nosecondary_final["Scaled LDL-C Max"]).sum()

807

In [17]:
ldlc_valid_18_nontg_index_nosecondary_final[ldlc_valid_18_nontg_index_nosecondary_final["LDL-C Max"] != ldlc_valid_18_nontg_index_nosecondary_final["Scaled LDL-C Max"]]

,Patient ID,LDL-C Max,Index Date,Scaled LDL-C Max
125,F7405823W,1.73,2015-11-01,2.768
128,F7475829X,2.71,2015-09-23,4.336
279,G1055848X,1.46,2018-08-21,2.336
368,G3056360W,2.74,2018-10-02,4.384
554,G6667404K,3.32,2018-01-20,6.640
...,...,...,...,...
20106,S9531639E,2.71,2015-07-08,4.336
20657,X0170018318H,2.72,2016-03-04,3.808
20699,X0170042553Z,2.91,2016-11-26,4.074
20714,X0170049475B,0.96,2017-11-02,1.728


In [18]:
##
def segment(ldlc_val):
    if ldlc_val >= 4.9:
        return "Cases"
    elif 3.36 < ldlc_val < 4.9:
        return "Unknown"
    elif ldlc_val <= 3.36:
        return "Control"
    else:
        raise ValueError("Unable to handle ldlc_val.")

ldlc_valid_18_nontg_index_nosecondary_final["Category"] = ldlc_valid_18_nontg_index_nosecondary_final["Scaled LDL-C Max"].apply(segment)

In [20]:
ldlc_valid_18_nontg_index_nosecondary_final["Category"].value_counts()

Category
Control    13947
Unknown     5572
Cases       1267
Name: count, dtype: int64

In [21]:
ldlc_valid_18_nontg_index_nosecondary_final.to_csv(
    os.path.join(PROJECT_ROOT, "results", "scaled_ldlc_valid_18_nontg_index_nosecondary_final.csv"),
    index=False,
)

In [22]:
ldlc_valid_18_nontg_index_nosecondary_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20786 entries, 0 to 20785
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Patient ID        20786 non-null  object        
 1   LDL-C Max         20786 non-null  float64       
 2   Index Date        20786 non-null  datetime64[ns]
 3   Scaled LDL-C Max  20786 non-null  float64       
 4   Category          20786 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 812.1+ KB


## End.